In [17]:
import torch
from PIL import Image
from create_model import IDOneClassClassifier
from albumentations.pytorch import ToTensorV2
import albumentations as A

ImportError: cannot import name 'IDOneClassClassifier' from 'create_model' (/Users/francobertoldi/Desktop/DNI-classifier/create_model.py)

In [ ]:
# Define transformation for the image (same as training)
transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# Load model parameters
device = torch.device('mps' if torch.mps.is_available() else 'cpu')
model_checkpoint_path = 'dataset/best_model_ResNet50_one_class.pth'

In [ ]:
checkpoint = torch.load(model_checkpoint_path, map_location=device)
model = IDOneClassClassifier().to(device)
model.load_state_dict(checkpoint['model_state_dict'])
center = checkpoint['center'].to(device)
radius = checkpoint['radius'].to(device)

model.eval()  # Set model to evaluation mode

In [ ]:
def preprocess_image(image_path):
    """
    Preprocess an image for the model.

    Args:
        image_path (str): Path to the image.

    Returns:
        torch.Tensor: Preprocessed image tensor.
    """
    img = Image.open(image_path).convert('RGB')
    img = transform(image=np.array(img))['image']
    img = img.unsqueeze(0)  # Add batch dimension
    return img.to(device)

def predict_image(image_path, model, center, radius, threshold_multiplier=1.0):
    """
    Predict if an image is normal or anomalous.

    Args:
        image_path (str): Path to the image to be tested.
        model (nn.Module): Loaded model.
        center (torch.Tensor): Center of the learned hypersphere.
        radius (torch.Tensor): Radius of the hypersphere.
        threshold_multiplier (float): Multiplier for threshold sensitivity.

    Returns:
        str: Prediction result ("normal" or "anomalous").
    """
    # Preprocess the image
    img = preprocess_image(image_path)

    # Get features and compute distance to center
    with torch.no_grad():
        features = model(img)
        distance = torch.sum((features - center) ** 2, dim=1)

    # Determine if the image is normal or anomalous
    is_normal = distance <= (radius * threshold_multiplier) ** 2
    return "normal" if is_normal.item() else "anomalous"

In [ ]:
# Test on an image
test_image_path = 'prueba/imagen_1_frente.jpeg'  # Replace with your image path
prediction = predict_image(test_image_path, model, center, radius)

print(f"The test image is predicted to be: {prediction}")